# Свёрточные коды

## Терминология

Линейный древовидный $(n_0, k_0)$ код с конечной длиной кодового ограничения называется свёрточным кодом. Кодер свёрточного кода может хранить $m$ ранее поступивших блоков из $k_0$ символов. Скорость кода так же равна $\frac{k_0}{n_0}$. Объём памяти кодера равна $K = k_0 \cdot m$.

По аналогии с линейными блоковыми кодами, для которых вводилась матрица переходов, для свёрточных кодов можно ввести граф переходов, соответствующий некоторому конечному автомату.

Также по аналогии с линейными блоковыми кодами, можно ввести понятие порождающей матрицы многочленов $\underset{k_0 \times n_0}{G(x)}$. Максимальная степень многочленов в порождающей матрице равна $m$.

### Катастрофические коды

Катастрофическим кодом называется код, отображающий информационные последовательности бесконечного веса в кодовые последовательности конечного веса. Ошибки в канале для катастрофического кода могут привести к бесконечному числу ошибок при декодировании.

Код не является катастрофическим тогда и только тогда, когда в графе переходов нет петель нулевого веса. Код не является катастрофическим тогда и только тогда, когда НОД определителей всех $k_0 \times k_0$ подматриц $G(x)$ равен $x^s$, где $s \ge 0$.

### Примеры

* $G(x) = (x^2 + 1, x^2 + x + 1)$ является некатастрофическим кодом.
* $G(x) = (\frac{x^2 + 1}{x^2 + x + 1}, 1)$ эквивалентен предыдущему.
* $G(x) = (x^2 + 1, x^2 + 1)$ является катастрофическим кодом.

## Минимальное расстояние

$i$-ым минимальным расстоянием кода $d_i^*$ называется минимально расстояние Хэмминга для любых последовательностей из $i$ кадров, отличающихся начальным кадром. $d_{m+1}^*$ называется минимальным расстоянием кода.

Последовательность $\{ d_i^* \}$ называется дистанционным профилем кода. Минимальным свободным расстоянием кода называется $d_{free} = \max_i{d_i^*}$.